In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import scipy.ndimage 
import shapely.geometry
import geocube.api.core 
import skimage.transform

import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy.random as random

import xycmap

# Overview
This notebook will read out data grouped by zip code and create a raster interpolation along with a 2D mask for a particular metropolitan statistical area

In the example here, the data is the number of patients evalulated for DBS from each zip code and the population within that zip code whose race is Black (both normalized against total population)
The notebook "download_acs_data.ipynb" has code to get data from the Census's American Community Survey


You will need a python environment with numpy, pandas, geopandas, scipy, shapely, geocube, scikit-image, matplotlib, statsmodels, and xycmap
All should be available on conda-forge or pip 

# 1. Data Loading

### Load the number of patients by zip code

The original data was the number of patients we considered for DBS, broken down by the zip code of their address of record

In [ ]:
pts = pd.read_csv('dbs_pts_byzip.csv').set_index('zip_code')
pts.head()

### Next load relevant demographic data, by zip code tabulation area 

This is public data provided by the Census (American Community Survey). 
Code to download it is included in "download_acs_data.ipynb". There are many types of demographic data available. It is broken down by zip code tabulation area, which is an approximation of the geographic encompassed by each zip code. 

In [ ]:
pop_total = pd.read_csv('pop_total.csv').set_index('zcta')
pop_total.head()

In [ ]:
pop_black = pd.read_csv('pop_black.csv').set_index('zcta')
pop_black.head()

### Combine the data into a single table and calculate per-capita metrics

In [ ]:
pts_pop = pop_total.join(pop_black).join(pts)
pts_pop = pts_pop[pts_pop['pop_total']>0]
pts_pop['pt_count'] = pts_pop['pt_count'].fillna(0).astype(int)

#the variables are normalized differently, for display
pts_pop['pct_black'] = (pts_pop['pop_black']/pts_pop['pop_total'])*100
pts_pop['pts_per_100k'] = (pts_pop['pt_count']/pts_pop['pop_total'])*100000

#reduce the data to the variables of interest
pts_pop = pts_pop[['pct_black','pts_per_100k']]

pts_pop.head()

### Next, add in geospatial data for each zip code tabulation area

In [ ]:
#Next, load the geometry of each zip code tabulation area
zipgeom = (gpd.read_file('tl_2020_us_zcta510/tl_2020_us_zcta510.shp')
            .rename(columns={'ZCTA5CE10':'zipcode','geometry':'zcta_polygon'}))[['zipcode','zcta_polygon']]
zipgeom['zipcode'] = zipgeom['zipcode'].astype(int)
zipgeom['zcta_centroid'] = [g.centroid for g in zipgeom['zcta_polygon']]
#Remove zip codes with no population
zipgeom = zipgeom.join(pop_total,on='zipcode')
zipgeom = zipgeom[zipgeom['pop_total']>0].drop(columns=['pop_total'])
zipgeom = zipgeom.set_geometry('zcta_polygon')
zipgeom.head()

In [ ]:
#attach the data of interest to the geospatial geometry
pts_pop_geom = zipgeom.join(pts_pop,on='zipcode')
pts_pop_geom.head()

# 2. Data processing

### Crop the geography to a region of interest

In [ ]:
#First, define the lat/lon of the center of our region of interest
roi_center_lon, roi_center_lat = -84.32179723327368,33.79198182080914

#These approximately convert lat/lon to km
km_per_lat = 110.574
km_per_lon = (111.320*np.abs(np.cos(roi_center_lat)))

#This determines the size of the roi to clip
#Should be a square centered at the roi_center extending "extent_km" in every direction
extent_km = 200
extent_lat = extent_km/km_per_lat
extent_lon = extent_km/km_per_lon


boundspoly = shapely.geometry.Polygon([ (roi_center_lon-extent_lon,roi_center_lat-extent_lat), 
                                        (roi_center_lon-extent_lon,roi_center_lat+extent_lat), 
                                        (roi_center_lon+extent_lon,roi_center_lat+extent_lat),
                                        (roi_center_lon+extent_lon,roi_center_lat-extent_lat)
                                      ])

pts_pop_geom_cropped = pts_pop_geom.clip(boundspoly)
pts_pop_geom_cropped.plot()

### Interpolate the data into a raster, smooth with gaussian filtering

In [ ]:
#Res/sigma will determine the resolution and degree of smoothing of the plot
#A high quality plot will be around res=.1,sigma=100 - this resolution is pretty computationally intensive
#can be reduced for a preview
res = .1
sigma = 10/res


#This creates the rasters for each variable
datatable = (pts_pop_geom_cropped
             .drop(columns=['zipcode'])
             .rename(columns={'zcta_polygon':'geometry'})
             .set_geometry('geometry'))
datacube = geocube.api.core.make_geocube(
                    vector_data=datatable,
                    resolution=(-res/km_per_lat, res/km_per_lon),
                    interpolate_na_method='cubic').fillna(0)

#This smooths the rasters for each variable
var_names = list(datacube.data_vars)
vars_interp = [datacube[vn].to_numpy() for vn in var_names]
vars_smooth = [scipy.ndimage.gaussian_filter(v,sigma) for v in vars_interp]


#Smoothed rasters are displayed below
fig,axes = plt.subplots(1,len(var_names))
for name,vs,ax in zip(var_names,vars_smooth,axes):
    ax.imshow(vs,cmap='Greens')
    ax.set_title(name)


In [ ]:
#The center of the roi (defined earlier) is calculated in raster space
roi_center_x = np.argmin(np.abs(datacube['x'].to_numpy()-roi_center_lon))
roi_center_y = np.argmin(np.abs(datacube['y'].to_numpy()-roi_center_lat))

### Create a mask for the Metropolitan statistical area of interest

In [ ]:
#This will use the Census code for the MSA/CBSA of interest
#12060 is metro Atlanta
msa_of_interest = 12060

#This is a file that contains a mapping between zip codes and MSA/CBSAs
zip_to_msa=pd.read_csv('ZIP_CBSA_122019.csv').set_index('ZIP')[['CBSA']]
zip_to_msa.index = zip_to_msa.index.astype(int)


#Here, we create a raster with a boolean value for whether each coordinate is in the MSA
#This is very similar to the raster of the population/patient variables, with some changes
#to accomodate a boolean
zips_in_msaoi = zip_to_msa[zip_to_msa['CBSA']==msa_of_interest].index.tolist()

zipgeom_msaoi = zipgeom.copy()
zipgeom_msaoi['in_msaoi'] = zipgeom['zipcode'].isin(zips_in_msaoi).astype(int)

zipgeom_msaoi_cropped = zipgeom_msaoi.clip(boundspoly)

msaoi_table = (zipgeom_msaoi_cropped
             .drop(columns=['zipcode'])
             .rename(columns={'zcta_polygon':'geometry'})
             .set_geometry('geometry'))
msaoi_cube = geocube.api.core.make_geocube(
                    vector_data=msaoi_table,
                    resolution=(-res/km_per_lat, res/km_per_lon),
                    interpolate_na_method='nearest')
msaoi_mask = msaoi_cube['in_msaoi'].to_numpy()


plt.imshow(msaoi_mask,cmap='gray')


# 3. Statistical analysis

### Process the rasters into geographic samples for statistics
This downscales the raster data such that there are an equal number of pixel samples as
there were zip codes, in the MSA of interest
This is used to avoid inflating the degrees of freedom for statistics

In [ ]:
smp_cnt_orig = np.sum(zipgeom_msaoi['in_msaoi'])
smp_cnt_working = np.sum(msaoi_mask)
ds = np.ceil(np.sqrt(np.sum(msaoi_mask)/smp_cnt_orig)).astype(int)
ds = 0 
#We have to optimize for the right downscaling factor, ds
while smp_cnt_working>smp_cnt_orig:
    ds = ds + 1
    msaoi_mask_ds = skimage.transform.downscale_local_mean(msaoi_mask,(ds,ds))>0
    smp_cnt_working = np.sum(msaoi_mask_ds)

vars_ds = [skimage.transform.downscale_local_mean(vs,(ds,ds)) for vs in vars_smooth]

#The sample count of our rasterified,smoothed, and downsampled data should be less than or 
#equal to the original number of geographic samples
ds, smp_cnt_working, smp_cnt_orig

### Run the regression, comparing two variables within these geographic samples

In [ ]:
var_names

In [ ]:
#Assign independent and dependent variables, and run a linear regression
y = vars_ds[1][msaoi_mask_ds].flatten()
X = vars_ds[0][msaoi_mask_ds].flatten()

linreg = sm.OLS(y, sm.add_constant(X)).fit()

#Plot scatterplot and display linear regression results
fig,ax = plt.subplots()
ax.scatter(X,y)
ax.set_xlim((0,100))
ax.set_ylim((0,10))

print(linreg.summary(xname=['intercept',var_names[0]],yname=var_names[1]))

linreg.pvalues


### Create a regression line

In [ ]:
reg_b, reg_m = linreg.params
reg_p = linreg.pvalues[1]
fit_x = np.array([0,100])
fit_y = fit_x*reg_m+reg_b

fig,ax = plt.subplots()
ax.scatter(X,y)
ax.set_xlim((0,100))
ax.set_ylim((0,10))
ax.plot(fit_x,fit_y,color='red',linestyle='dashed')
ax.text(80,1.4,f'p={reg_p:1.4f}',color='red')

# Plotting

### Create a colormap using xycmap
this is fairly computationally intensive at high resolution (~10min to run)

In [ ]:
#Here we pull out our variables (shown above) into x and y
sx = vars_smooth[0].flatten()
sy = vars_smooth[1].flatten()
smask = msaoi_mask.flatten()

#Here we define the axis limits for each variable
#For instance, here x is a percentage
axlims = {'xlims':(0,100)}

#This defines aspects of the colormap
#Uses the xycmap package
corner_colors = ("lightgrey", "blue", "red", "purple")
n = (7, 7) 

#This creates the colormap, applies it to both variables, and reshapes it into an image
cmap = xycmap.custom_xycmap(corner_colors=corner_colors, n=n)
clist = xycmap.bivariate_color(sx=sx, sy=sy, cmap=cmap,**axlims)
cimg = np.array(clist.to_list()).reshape(*vars_smooth[0].shape,4)

fig,axes = plt.subplots(1,2)
axes[0].imshow(cimg)
xycmap.bivariate_legend(ax=axes[1], sx=sx, sy=sy, cmap=cmap,**axlims)

### Create an MSA mask with smooth borders and apply it

In [ ]:
border_gsigma = 6
border_dilations = 90
msaoi_mask_smooth = scipy.ndimage.gaussian_filter(msaoi_mask.astype(float),border_gsigma)
msaoi_mask_smooth = scipy.ndimage.binary_dilation(msaoi_mask,iterations=border_dilations).astype(float)
plt.imshow(msaoi_mask_smooth)

In [ ]:
cimg_msaoi = cimg.copy()
cimg_msaoi[msaoi_mask_smooth==0] = (1,1,1,0)
plt.imshow(cimg_msaoi)

### Crop the image to the foreground

In [ ]:
fg_indices = np.nonzero(msaoi_mask_smooth)
#This finds the first/last pixel that is not maxed out, in both dimensions
xmin,xmax = np.flatnonzero(msaoi_mask_smooth.any(0))[[0,-1]]
ymin,ymax = np.flatnonzero(msaoi_mask_smooth.any(1))[[0,-1]]

cimg_msaoi_cropped = cimg_msaoi.copy()
cimg_msaoi_cropped = cimg_msaoi_cropped[ymin:ymax,xmin:xmax]

plt.imshow(cimg_msaoi_cropped)

### Create the final plot

In [ ]:
fig,geoax = plt.subplots()
plotax = fig.add_axes([0.28, -.3, 0.26, 0.3])
cax = fig.add_axes([0.600, -.20, 0.25, 0.25])

dred = np.array((119,1,17,255))/255

#Plot the geospatial imgage
geoax.imshow(cimg_msaoi_cropped)
geoax.plot(roi_center_x-xmin,roi_center_y-ymin,
                color='gold',markeredgecolor='black',marker='*', 
                markersize=25,zorder=101)

geoax.axis('off')

#Plot the colormap legend
cax = xycmap.bivariate_legend(ax=cax, sx=sx, sy=sy, cmap=cmap,xlims=(0,100))
cax.spines['right'].set_visible(False)
cax.spines['top'].set_visible(False)
cax.spines['left'].set_visible(False)
cax.spines['bottom'].set_visible(False)
cax.set_xticks(cax.get_xlim())
cax.set_xticklabels([0,100])
cax.set_yticks(cax.get_ylim())
cax.set_yticklabels([0,15])
cax.set_ylabel('DBS Patients/100k',fontsize=10)
cax.set_xlabel('% Population Black',fontsize=10)


#Plot the scatterplot of geographic samples used for stats
plotax.scatter(X,y,color='black',s=1.5)
plotax.set_xlim((0,100))
plotax.plot(fit_x,fit_y,color=dred,marker=None,linestyle='solid')
plotax.text(50,4,f'p={reg_p:1.4f}',color=dred)

plotax.set_xlabel('% Population Black',fontsize=10)
plotax.set_ylabel('DBS Patients/100K',fontsize=10)

plotax.spines['right'].set_visible(False)
plotax.spines['top'].set_visible(False)


fig.subplots_adjust(wspace=0, hspace=0,bottom=0,top=1,left=0, right=1) 
fig.tight_layout()

#This labels each subplot
laxes = [geoax,plotax]
labels = ['A','B'] 
offsets = [(0,200),(-.4,11)]
for label,offset,ax in zip(labels,offsets,laxes):
    ax.text(offset[0],
            offset[1],
            label,
            fontweight='bold',
            color='black',backgroundcolor='white')
    
fig.savefig('georaster_plot.png',dpi=1000,transparent=False,bbox_inches='tight',facecolor='white')